In [1]:
#!wget https://gist.githubusercontent.com/lazaros05/a536628fe1f626e0573743|1d2237b7cc/raw/63696c866a4a64569f814ceca2ab26b3d44bbe0c/BC_cardiotox_clinical_variables.csv
!rm creditcard.csv
!curl https://gist.githubusercontent.com/lazaros05/7a5d126c8c65b0b083a7146bc886f543/raw/8703c449d7913ac406f412e1b74dea6a85287422/creditcard.b64 | base64 -d > a.zip
!unzip a.zip
!sed -i 's/"0"/0/g' creditcard.csv
!sed -i 's/"1"/1/g' creditcard.csv
# !pip install -U imbalanced-learn==0.10.1

rm: cannot remove 'creditcard.csv': No such file or directory
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 87.9M  100 87.9M    0     0  39.9M      0  0:00:02  0:00:02 --:--:-- 39.9M
Archive:  a.zip
  inflating: creditcard.csv          


In [2]:
import numpy as np
import pandas as pd

# Load the file directly into a NumPy array
_data = pd.read_csv("creditcard.csv",sep=',', header=0, decimal=".")
_data.drop_duplicates(inplace=True)

def process_column(col):
    dataset_1_col = col.astype(str).str.split(',').str[0].astype(float)
    dataset_2_col = col.astype(str).str.replace(',', '.', regex=False).astype(float)
    return dataset_1_col, dataset_2_col

dataset_2 = pd.DataFrame()

# Choose columns that contain comma-like values
target_columns = [col for col in _data.columns if _data[col].astype(str).str.contains(',').any()]

for col in target_columns:
    col1, col2 = process_column(_data[col])
    dataset_2[col] = col2
for col in dataset_2.columns:
    _data[col] = dataset_2[col]
_data = _data.fillna(0)


import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import MinMaxScaler

# Assuming your DataFrame is called `data`

# 1. Drop the `ID` column
# _data = _data.drop(columns=["ID"])




from sklearn.preprocessing import PowerTransformer, QuantileTransformer
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_curve
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler ,MinMaxScaler , Normalizer
from torch.nn.functional import normalize
from torch.utils.data import Dataset, DataLoader , TensorDataset
import torch
import torch.nn as nn
import math
from functools import reduce
import builtins
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
import pandas as pd
import numpy as np


import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from imblearn.over_sampling import SMOTE

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score ,confusion_matrix,classification_report
from tensorflow.keras import layers
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
import xgboost as xgb

import joblib

import time

In [4]:


  seed = 0

  data = _data.copy()

  # data poisoning
  np.random.seed(32 + seed)
  num_columns = data.shape[1] - 1
  random_indices = np.random.choice(range(num_columns), size=10, replace=False)
  for idx in random_indices:
    data.iloc[:, idx] = np.random.normal(loc=0, scale=1, size=len(data))
  # indices
  all_indices = set(range(num_columns))  # All possible column indices except the last one
  non_random_indices = sorted(all_indices - set(random_indices))
  # scaling/split
  X = data.drop('Class',axis=1)
  y = data['Class']

  X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
  X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.125, random_state=42, stratify=y_temp)

  scaler_X = MinMaxScaler(feature_range=(0,1))
  X_train = scaler_X.fit_transform(X_train)
  X_val  = scaler_X.transform(X_val)
  X_test  = scaler_X.transform(X_test)
  joblib.dump(scaler_X,f'Scaling.joblib')
  smote = SMOTE(random_state=42,sampling_strategy=0.1) #minority is 0.3 from majority
  X_train, y_train = smote.fit_resample(X_train, y_train)

  pt = PowerTransformer(method="yeo-johnson")
  X_train = pt.fit_transform(X_train)
  X_val = pt.transform(X_val)
  X_test = pt.transform(X_test)

  scaler = MinMaxScaler()
  X_train_scaled = scaler.fit_transform(X_train)
  X_val_scaled = scaler.transform(X_val)
  X_test_scaled = scaler.transform(X_test)

  print(y_test.value_counts())
  print(y_val.value_counts())
  print(y_train.value_counts())

  # quantum model definitions

  class BinaryClassifierFRAUD(nn.Module):
      def __init__(self, input_dim):
          torch.manual_seed(1234 + seed)
          super(BinaryClassifierFRAUD, self).__init__()
          # self.net = nn.Sequential(
          #     nn.Linear(input_dim, 16),
          #     nn.ReLU(),
          #     nn.Linear(16, 8),
          #     nn.ReLU(),
          #     nn.Linear(8, 1)  # output raw logits (no sigmoid)
          # )
          self.net = nn.Sequential(
              nn.Linear(input_dim, 32),
              nn.ReLU(),
              nn.Linear(32, 32),
              nn.ReLU(),
              nn.Linear(32, 32),
              nn.ReLU(),
              nn.Linear(32, 32),
              nn.ReLU(),
              nn.Linear(32, 16),
              nn.ReLU(),
              nn.Linear(16, 8),
              nn.ReLU(),
              nn.Linear(8, 1)  # output raw logits (no sigmoid)
          )

      def forward(self, x):
          return self.net(x)

  # prepare for training
  nn_model = BinaryClassifierFRAUD(30)
  print(sum(p.numel() for p in nn_model.parameters() if p.requires_grad))
  criterion = nn.BCEWithLogitsLoss()
  optimizer = optim.Adam(nn_model.parameters(), lr=0.01)
  num_epochs = 100
  batch_size = 4
  # labels = np.eye(6)[y_train.values]
  labels = y_train.values
  dataset_train = TensorDataset(torch.tensor(X_train_scaled) , torch.tensor(labels))
  data_loader_train = DataLoader(dataset_train,batch_size= batch_size,shuffle=True)
  # labels_test = np.eye(6)[y_test.values]
  labels_test = y_test.values
  dataset_test = TensorDataset(torch.tensor(X_test_scaled) , torch.tensor(labels_test) )
  data_loader_test = DataLoader(dataset_test,batch_size= batch_size,shuffle=True)
  labels_val = y_val.values
  dataset_val = TensorDataset(torch.tensor(X_val_scaled), torch.tensor(labels_val))
  data_loader_val = DataLoader(dataset_val, batch_size=batch_size, shuffle=True)
  # training
  # Set the L1 regularization strength (lambda)
  lambda_l1 = 0.00001
  training_threshold = 0.0002
  prev_loss = torch.inf
  no_decr = 0
  epoch_threshold = 5
  # Training loop
  val_loss_list = list()
  train_loss_list = list()
  for epoch in range(num_epochs):
      start_time = time.perf_counter()
      nn_model.train()
      total_loss = 0.0
      for inputs, labels in data_loader_train:
          optimizer.zero_grad()
          # Forward pass
          outputs = nn_model(inputs.float())
          # Compute the primary loss
          labels = labels.to(float)
          labels = labels.view(-1, 1)
          # print(outputs)
          # print(labels)
          loss = criterion(outputs, labels)
          # Add L1 regularization loss
          l1_loss = 0.0
          for param in nn_model.parameters():
              l1_loss += torch.sum(torch.abs(param))  # Sum of absolute values of weights
          # loss += lambda_l1 * l1_loss  # Add L1 penalty to the primary loss
          # Backward pass
          loss.backward()
          # Update weights
          optimizer.step()
          total_loss += loss.item()
      # Print the average loss for each epoch
      average_loss = total_loss / len(data_loader_train)
      # print(f'Epoch {epoch + 1:3d}/{num_epochs}, Loss: {average_loss}')
      nn_model.eval()
      val_loss_total = 0.0
      with torch.no_grad():
          for val_inputs, val_labels in data_loader_val:
              val_outputs = nn_model(val_inputs.float())
              val_labels = val_labels.to(torch.float)
              val_labels = val_labels.view(-1, 1)
              val_loss = criterion(val_outputs, val_labels)
              val_loss_total += val_loss.item()
      average_val_loss = val_loss_total / len(data_loader_val)
      if average_val_loss + training_threshold > prev_loss:
          no_decr += 1
          if no_decr == epoch_threshold:
              nn_model.load_state_dict(torch.load(str(seed) + 'model_weights.pth'))
              print(f'Epoch {epoch + 1:3d}/{num_epochs}, '
                    f'Train Loss: {average_loss:.4f}, '
                    f'Val Loss: {average_val_loss:.4f} '
                    f'Inc Streak: {no_decr}')
              print('---Stopping training---')
              val_loss_list.append(average_val_loss)
              train_loss_list.append(average_loss)
              break
      else:
          torch.save(nn_model.state_dict(), str(seed) + 'model_weights.pth')
          no_decr = 0
          prev_loss = average_val_loss
      print(f'Epoch {epoch + 1:3d}/{num_epochs}, '
            f'Train Loss: {average_loss:.4f}, '
            f'Val Loss: {average_val_loss:.4f} '
            f'Inc Streak: {no_decr}')
      val_loss_list.append(average_val_loss)
      train_loss_list.append(average_loss)
      end_time = time.perf_counter()
      print(f'Time taken: {end_time - start_time:.2f} seconds')






  # evaluate quantum model
  nn_model.eval() # Set the model to evaluation mode
  correct_predictions = 0
  total_samples =0
  with torch.no_grad():
    for inputs, labels in data_loader_test:
      outputs = torch.sigmoid(nn_model(inputs.float()).float())
      # _, predicted = torch.max(outputs,1)
      predicted = (outputs > 0.5).float()
      total_samples += labels.size(0)
      correct_predictions += (predicted.squeeze() == labels).sum().item()
      # correct_predictions += (predicted == labels.argmax(dim=1)).sum().item()
  accuracy = correct_predictions / total_samples
  nn_results = (accuracy,)
  print(f'Accuracy on the training set: {accuracy*100:.2f}%')
  nn_model.eval()  # Set the model to evaluation mode

  all_predictions = []  # To store all predictions
  all_labels = []       # To store all actual labels

  with torch.no_grad():
      for inputs, labels_test in data_loader_test:
          # outputs = nn_model(inputs.float())
          outputs = torch.sigmoid(nn_model(inputs.float()).float())
          # _, predicted = torch.max(outputs, 1)  # Get the predicted class indices
          predicted = (outputs > 0.5).float()
          # Collect predictions and labels
          all_predictions.extend(predicted.squeeze().cpu().numpy())  # Convert to numpy array and append
          # all_labels.extend(labels_test.argmax(dim=1).cpu().numpy())  # Convert labels to numpy and append
          all_labels.extend(labels_test.cpu().numpy())  # Convert labels to numpy and append
  # classification report


  predictions_series = pd.Series(all_predictions)
  labels_series = pd.Series(all_labels)
  predictions_series = predictions_series.clip(upper=1)
  report = classification_report(labels_series, predictions_series)
  nn_results = nn_results + (classification_report(labels_series, predictions_series, output_dict=True),)
  print("\nClassification Report:\n")
  print(report)
  # feature importance of quantum
  # Prepare a dictionary to store parameter row sums
  nn_results = nn_results + ((0, 0),)
  nn_results = nn_results + ((train_loss_list, val_loss_list,),)

  results = (("logistic_regression_results",),) + ((nn_results,),) + (("rf_results",),)


  # nn_dict = {
  #     # '% of matched features' : [results[i][1][0][2][0] for i in range(len(seeds))],
  #     # 'weight sum/diff score' : [results[i][1][0][2][1] for i in range(len(seeds))],
  #     'class 0, precision' : [results[i][1][0][1]['0']['precision'] for i in range(len(seeds))],
  #     'class 0, recall' : [results[i][1][0][1]['0']['recall'] for i in range(len(seeds))],
  #     'class 0, f1-score' : [results[i][1][0][1]['0']['f1-score'] for i in range(len(seeds))],
  #     'class 0, support' : [results[i][1][0][1]['0']['support'] for i in range(len(seeds))],
  #     'class 1, precision': [results[i][1][0][1]['1']['precision'] for i in range(len(seeds))],
  #     'class 1, recall': [results[i][1][0][1]['1']['recall'] for i in range(len(seeds))],
  #     'class 1, f1-score': [results[i][1][0][1]['1']['f1-score'] for i in range(len(seeds))],
  #     'class 1, support': [results[i][1][0][1]['1']['support'] for i in range(len(seeds))],
  #     'accuracy': [results[i][1][0][1]['accuracy'] for i in range(len(seeds))],
  #     'macro avg precision': [results[i][1][0][1]['macro avg']['precision'] for i in range(len(seeds))],
  #     'macro avg recall': [results[i][1][0][1]['macro avg']['recall'] for i in range(len(seeds))],
  #     'macro avg f1-score': [results[i][1][0][1]['macro avg']['f1-score'] for i in range(len(seeds))],
  #     'macro avg support': [results[i][1][0][1]['macro avg']['support'] for i in range(len(seeds))],
  #     'weighted avg precision': [results[i][1][0][1]['weighted avg']['precision'] for i in range(len(seeds))],
  #     'weighted avg recall': [results[i][1][0][1]['weighted avg']['recall'] for i in range(len(seeds))],
  #     'weighted avg f1-score': [results[i][1][0][1]['weighted avg']['f1-score'] for i in range(len(seeds))],
  #     'weighted avg support': [results[i][1][0][1]['weighted avg']['support'] for i in range(len(seeds))],
  #     'train_loss' : [results[i][1][0][3][0] for i in range(len(seeds))],
  #     'val_loss' : [results[i][1][0][3][1] for i in range(len(seeds))]
  # }

  # # lr_dt = pd.DataFrame(lr_dict)
  # # rf_dt = pd.DataFrame(rf_dict)
  # nn_dt = pd.DataFrame(nn_dict)

  # # lr_dt.to_csv("lr_fraud.csv")
  # # rf_dt.to_csv("rf_fraud.csv")
  # nn_dt.to_csv("nn_fraud.csv")

Class
0    56651
1       95
Name: count, dtype: int64
Class
0    28326
1       47
Name: count, dtype: int64
Class
0    198276
1     19827
Name: count, dtype: int64
4833
Epoch   1/100, Train Loss: 0.3050, Val Loss: 0.1115 Inc Streak: 0
Time taken: 150.93 seconds
Epoch   2/100, Train Loss: 0.3050, Val Loss: 0.0978 Inc Streak: 0
Time taken: 154.86 seconds
Epoch   3/100, Train Loss: 0.3050, Val Loss: 0.0883 Inc Streak: 0
Time taken: 153.61 seconds
Epoch   4/100, Train Loss: 0.3050, Val Loss: 0.0965 Inc Streak: 1
Time taken: 151.62 seconds
Epoch   5/100, Train Loss: 0.3050, Val Loss: 0.0967 Inc Streak: 2
Time taken: 155.83 seconds
Epoch   6/100, Train Loss: 0.3050, Val Loss: 0.0911 Inc Streak: 3
Time taken: 155.72 seconds
Epoch   7/100, Train Loss: 0.3050, Val Loss: 0.0954 Inc Streak: 4
Time taken: 152.12 seconds
Epoch   8/100, Train Loss: 0.3050, Val Loss: 0.1073 Inc Streak: 5
---Stopping training---
Accuracy on the training set: 99.83%

Classification Report:

              precision    r

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/m